In [1]:
import os
import sys
import time
import logging
import warnings
from collections import defaultdict, Counter
from tqdm import tqdm
import sqlite3 as sql
from sqlalchemy import create_engine
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np


In [2]:
def prep_image_table(df_image, plate_wells):
    # Getting the well and field id information straight away
    df_image['Well_ID'] = [i[:3] for i in df_image['Image_FileName_CellOutlines']]
    df_image['Field_ID'] = [i[:6] for i in df_image['Image_FileName_CellOutlines']]
    # subsetting the dataframe to include only relevant well ids, based on the 
    # endpoint annotated dataset 
    df_image = df_image[df_image['Well_ID'].isin(plate_wells)]
    return df_image


def query_type(object_type):
    # creating custom queries based on the cell/object type

    if object_type == 'Cytoplasm':
        query= (
                "SELECT * "
                "FROM Cytoplasm "
                f"WHERE TableNumber IN ({','.join(['?'] * len(table_number_plate))});"
        )
    elif object_type == 'Nuclei':
        query= (
                "SELECT * "
                "FROM Nuclei "
                f"WHERE TableNumber IN ({','.join(['?'] * len(table_number_plate))});"
        )
    else: 
        query= (
                "SELECT * "
                "FROM Cells "
                f"WHERE TableNumber IN ({','.join(['?'] * len(table_number_plate))});"
        )
        
    return query

def clean_object_df(object_df):
    # coercing all object columns (except the first one, TableNumber) to numeric
    object_cols = object_df.select_dtypes(exclude="number").columns[1:].tolist()
    object_df[object_cols] = object_df[object_cols].apply(pd.to_numeric, errors='coerce')

    # downcasting floats and integers to reduce the df size
    for col in object_df.select_dtypes(include=['int64', 'float64']).columns:
        if pd.api.types.is_integer_dtype(object_df[col]):
            object_df[col] = object_df[col].astype('int32')
        elif pd.api.types.is_float_dtype(object_df[col]):
            object_df[col] = object_df[col].astype('float16')

    return object_df

In [ ]:
meta_data = r"P:\Main\364 Image based phenotypic characterization\Emre's project\Data\603_compounds_metadata.xlsx"
base_path = r"P:\Main\364 Image based phenotypic characterization\Emre's project\Data\bray2017\sqlite"
save_dir = r"P:\Main\364 Image based phenotypic characterization\Emre's project\Data\parquet"

plates = [24279, 24280, 24293, 24294, 24295, 24296, 24300, 24301, 24302, 24303]
all_dfs = []

for plate in plates:
    # loading the metadata to know which wells from each plate are relevant 
    meta_data_plate = pd.read_excel(meta_data, usecols=['Metadata_Plate', 'Metadata_Well'])
    meta_data_plate = meta_data_plate[meta_data_plate['Metadata_Plate']==plate]
    plate_wells = meta_data_plate['Metadata_Well'].tolist()


    sub_folder = f'Plate_{plate}\\extracted_features\\{plate}.sqlite'
    db_path = os.path.join(base_path, sub_folder)
    conn = sql.connect(db_path)
    query = 'SELECT TableNumber, Image_FileName_CellOutlines FROM Image;'

    # Getting only the relevant information form the Image table
    df_image = pd.read_sql_query(query, conn)
    df_image = prep_image_table(df_image, plate_wells)
    df_image['PlateID'] = plate
    # Using  the remaining table number ids to subset cytoplasm, cells and nuclei tables
    table_number_plate = df_image['TableNumber'].tolist()

    # deleting any dfs to clear memory, just in case 
    del meta_data_plate

    for i, object_type in enumerate(['Cytoplasm', 'Nuclei', 'Cells']):
        query = query_type(object_type)

        # loading each table with only relevant table number ids 
        object_df = pd.read_sql_query(query, conn, params=table_number_plate)
        print(f"before downcasting the {object_type}, plate {plate}")
        print(object_df.info())
        object_df_cleaned = clean_object_df(object_df)
        print(f"after downcasting the {object_type}, plate {plate}")
        print(object_df_cleaned.info())
        #parquet_path = os.path.join(save_dir, f'clean_data{object_type}_{plate}.parquet')

        # merging the cell/object tables for each plate 
        if i == 0:
            df_object_combined = object_df_cleaned
            del object_df_cleaned
        else:
            df_object_combined = pd.merge(df_object_combined, object_df_cleaned, 
                                          on =['TableNumber', 'ImageNumber', 'ObjectNumber'])
            del object_df_cleaned
        #object_df_cleaned.to_parquet(parquet_path)
    df_plate = pd.merge(df_image, df_object_combined, on='TableNumber', how='right')
    parquet_path_plate = os.path.join(save_dir, f'clean_data_{plate}.parquet')
    df_plate.to_parquet(parquet_path_plate)
    print(f'The size of merged df, plate {plate} is')
    print(df_plate.info())
    all_dfs.append(df_plate)
    del df_plate
    conn.close()

    

before downcasting the Cytoplasm, 24279
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61039 entries, 0 to 61038
Columns: 585 entries, TableNumber to Cytoplasm_Texture_Variance_RNA_5_0
dtypes: float64(368), int64(6), object(211)
memory usage: 272.4+ MB
None
after downcasting the Cytoplasm, 24279
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61039 entries, 0 to 61038
Columns: 585 entries, TableNumber to Cytoplasm_Texture_Variance_RNA_5_0
dtypes: float16(578), int32(6), object(1)
memory usage: 69.2+ MB
None
before downcasting the Nuclei, 24279
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61039 entries, 0 to 61038
Columns: 608 entries, TableNumber to Nuclei_Texture_Variance_RNA_5_0
dtypes: float64(587), int64(8), object(13)
memory usage: 283.1+ MB
None
after downcasting the Nuclei, 24279
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61039 entries, 0 to 61038
Columns: 608 entries, TableNumber to Nuclei_Texture_Variance_RNA_5_0
dtypes: float16(599), int32(8), object(1)
memor

In [4]:
df_final = pd.concat(all_dfs)

In [ ]:
# df_final.to_parquet(r"P:\Main\364 Image based phenotypic characterization\Emre's project\Data\parquet\10_plates_cleaned_data.parquet")